In [ ]:
# Final Revision of Source Data Merge

In [ ]:
# Data Merge File Notes:
# File Sequence # 3

# The purpose of this notebook is to merge the following datasets:

# Inputs:
# A. Raw Stock Data
# B. Raw Economic Indicator Data

# ... in preparation for metrics the Stock KPI Measurements Notebook

# Output: Merged Stock and Economic Indicator Data


In [ ]:
# mount google drive
from google.colab import drive
# this resets all file variables
drive.flush_and_unmount()
# mount/remount
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
# import libraries:

# data handling
import pandas as pd
# file handling
import os


In [ ]:
# define various notebook objects

# define filtered date range for final output dataframe

#(many of the economic KPIs begin in 1992)
beg_date = '1992-06-01'

# end date, short of present, but still 'modern':
end_date = '2025-01-15'

# define shared directory:
shared_directory = '/content/drive/MyDrive/Capstone_Docs_Shared/'

In [ ]:
# define function : load and chcek nans
def load_pth_chk_nans(share_direct, filename):
  # full filepath:
  filepath = os.path.join(share_direct, filename)
  print(f"Selected Doc Found: {filepath}")
  # load data:
  df = pd.read_csv(filepath)
  # print nan counts:
  print("Nan Counts in Selected Doc:")
  print(df.isna().sum())
  return df


In [ ]:
# make actual data load for stock data and econ data:

# stock data first:
stock_data = load_pth_chk_nans(shared_directory, "merged_hist_stock_output.csv")
#print(stock_data)

# econ data next:
econ_data = load_pth_chk_nans(shared_directory, "unscaled_indicator_data.csv")
#print(econ_data)


Selected Doc Found: /content/drive/MyDrive/Capstone_Docs_Shared/merged_hist_stock_output.csv
Nan Counts in Selected Doc:
Date                0
Ticker              0
Open                0
High                0
Low                 0
Close               0
Volume        2767216
Adj Close    15364705
dtype: int64
Selected Doc Found: /content/drive/MyDrive/Capstone_Docs_Shared/unscaled_indicator_data.csv
Nan Counts in Selected Doc:
observation_date    0
UMCSENT             0
DGORDER             0
GDP                 0
PCE                 0
HOUST               0
RSXFS               0
ICSA                0
T10Y2Y              0
PPIACO              0
ADXDNO              0
dtype: int64


In [ ]:
# filter stock data based on defined date limits:

# perform date filter
stock_data = stock_data[(stock_data['Date'] >= beg_date) & (stock_data['Date'] <= end_date)]

# drop Adj_Close column
stock_data = stock_data.drop(columns=['Adj Close'])
# (Adj_Close is dropped because it introduces data leakage)

# Problem -- Adj Close introduces Data Leakage
# Cause -- Adj Close takes into account:
# future stock splits, reverse splits, dividends...
# ... ALL from present day backward...
# ... therefore (e.x. 2009 splits are reflected in 1993 Adj Close price)...
# ALSO: Adj Close Returns untethered from Close Returns

# Solution: Just use 'Close'
# 'Close' == "Price that traders saw on that day"

# sort by date
stock_data = stock_data.sort_values(by=['Date'])

# reset index
stock_data = stock_data.reset_index(drop=True)

print(stock_data)

                Date Ticker        Open        High         Low       Close  \
0         1992-06-01    AES    3.084969    3.137703    3.084969    3.137703   
1         1992-06-01    CDE  145.063814  148.660438  145.063814  147.461563   
2         1992-06-01    CPK    2.237710    2.475765    2.237710    2.475765   
3         1992-06-01    CRK    5.603199    5.883359    5.603199    5.603199   
4         1992-06-01    CRS    5.140174    5.181516    5.140174    5.181516   
...              ...    ...         ...         ...         ...         ...   
15612912  2025-01-15    WTS  206.580700  207.568700  202.459061  204.165604   
15612913  2025-01-15   XPER    9.140000    9.220000    8.995000    9.030000   
15612914  2025-01-15   YETI   38.570000   38.970001   36.860001   36.869999   
15612915  2025-01-15    YMM   11.300000   11.350000   11.050000   11.140000   
15612916  2025-01-15    ZTR    5.620865    5.689056    5.581899    5.689056   

             Volume  
0          420858.0  
1      

In [ ]:
# merge econ and stock data

# use left merge to preserve structure of stock_data, fill with econ_data info
merged_df = pd.merge(stock_data, econ_data, left_on='Date', right_on='observation_date', how='left')

# drop observation_date
merged_df = merged_df.drop(columns=['observation_date'])

print(merged_df)

print(merged_df.columns)

                Date Ticker        Open        High         Low       Close  \
0         1992-06-01    AES    3.084969    3.137703    3.084969    3.137703   
1         1992-06-01    CDE  145.063814  148.660438  145.063814  147.461563   
2         1992-06-01    CPK    2.237710    2.475765    2.237710    2.475765   
3         1992-06-01    CRK    5.603199    5.883359    5.603199    5.603199   
4         1992-06-01    CRS    5.140174    5.181516    5.140174    5.181516   
...              ...    ...         ...         ...         ...         ...   
15612912  2025-01-15    WTS  206.580700  207.568700  202.459061  204.165604   
15612913  2025-01-15   XPER    9.140000    9.220000    8.995000    9.030000   
15612914  2025-01-15   YETI   38.570000   38.970001   36.860001   36.869999   
15612915  2025-01-15    YMM   11.300000   11.350000   11.050000   11.140000   
15612916  2025-01-15    ZTR    5.620865    5.689056    5.581899    5.689056   

             Volume  UMCSENT   DGORDER        GDP  

In [ ]:
# NEW EXPORT BY DATE SPLIT: REV2

def split_and_export(input_df, num_desired_splits, shared_direct):
# need to split merged df into multiple chunks for ease of processing

  # take list of unique dates
  date_list = merged_df['Date'].unique()

  # take number of unique dates
  num_days = len(merged_df['Date'].unique())
  print('Total Number of Days: ', num_days)

  # SPLIT INTO MULTI+ FILES
  num_file_splits = num_desired_splits
  split_size = num_days // num_file_splits
  remainder = num_days % num_file_splits
  print('Split Size: ', split_size)
  print('Remainder: ', remainder)

  # define where to start count for split
  start_idx = 0

  # initialize loop to index splits:
  for i in range(num_file_splits):
    # set end_idx, (add one if there are remainders left)
    end_idx = start_idx + split_size + (1 if i < remainder else 0)
    #print(start_idx, end_idx) $MC

    # define date range
    date_range = date_list[start_idx:end_idx]
    #print(date_range) $MC

    # take subset of databframe based on date range
    data_window = merged_df[merged_df['Date'].isin(date_range)]

    # create new file name
    file_name = "source_data_merged_{}.csv".format(i+1)

    # define output path
    output_path = os.path.join(shared_direct, file_name)

    # export to csv
    data_window.to_csv(output_path, index=False)
    # (index=False prevents Unnamed: 0 Column problem)

    # increment start_idx
    start_idx = end_idx

    # print a visual progress statement
    print(f"Number {i+1} of {num_file_splits} complete")

  # at the end, print a success message
  print("Source Data Split by Date and Exported")

In [ ]:
# Execute file split

# Define desired number of splits:
num_splits = 10
# function call
split_and_export(merged_df, num_splits, shared_directory)

Total Number of Days:  8216
Split Size:  821
Remainder:  6
Number 1 of 10 complete
Number 2 of 10 complete
Number 3 of 10 complete
Number 4 of 10 complete
Number 5 of 10 complete
Number 6 of 10 complete
Number 7 of 10 complete
Number 8 of 10 complete
Number 9 of 10 complete
Number 10 of 10 complete
Source Data Split by Date and Exported
